In [1]:
import urllib
from urllib.error import HTTPError
import requests
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import csv
from xml import etree
from xml.sax.saxutils import unescape
import xml.etree.ElementTree as ET
import json
import html
import pandas as pd
import numpy as np
import re
import unicodedata
import datetime

In [2]:
pd.options.mode.chained_assignment = None

In [3]:
def get_root(url):
    req = urllib.request.Request(url=url)
    try:
        handler = urllib.request.urlopen(req)
        tree = ET.ElementTree(file = handler)
        root = tree.getroot()
        return root
    except HTTPError as e:
        content = e.read()
        return content

In [4]:
base_url = "https://www.costa-coffee.cz/express.php"

In [5]:
root = get_root(base_url)  

In [6]:
def get_column_names(root):
    tags = []
    for i, child in enumerate(root):  
        for subchild in child[-1:]: 
            for k in subchild:
                tags.append(k.tag)
    return tags

In [7]:
tags = get_column_names(root)

In [8]:
def get_data(root):
    records = []
    data = []
    for i, child in enumerate(root):
        for subchild in child:
            for j in subchild:
                records.append(j.text)
            try:
                if len(records) != 0:
                    data.append(records)
                    records = []
            except AttributeError as error:
                return error
    return data

In [9]:
records = get_data(root)

In [10]:
rawDf = pd.DataFrame(records, columns=tags)

In [12]:
rawDf.head()

,title,category,lat,lng,adresa,guid
0,Costa Express - Praha 4,Costa Express,50.034589163874585,14.455249451931763,"ČS Shell - Praha 4, ulice Vídeňská",https://www.costa-coffee.cz/costa/costa-express
1,Costa Express - Mělník,Costa Express,50.34802581272294,14.492174487249713,"ČS Shell - Mělník, ulice Mladoboleslavská",https://www.costa-coffee.cz/costa/costa-express
2,Costa Express - Praha 6,Costa Express,50.093505335502975,14.331827076074205,"ČS Shell - Praha 6, ulice Evropská",https://www.costa-coffee.cz/costa/costa-express
3,Costa Express - Rokycany,Costa Express,49.7538998,13.5894548,"ČS Shell - Rokycany, dálnice D5 (směr Praha)",https://www.costa-coffee.cz/costa/costa-express
4,Costa Express - Tábor,Costa Express,49.41056965127617,14.683676139550812,"ČS Shell - Tábor, ulice Chýnovská",https://www.costa-coffee.cz/costa/costa-express


In [13]:
expressDf = rawDf[["title", "lat", "lng", "adresa"]]

In [15]:
expressDf = expressDf.rename(columns={'adresa': 'address'})

In [17]:
expressDf.head()

,title,lat,lng,address
0,Costa Express - Praha 4,50.034589163874585,14.455249451931763,"ČS Shell - Praha 4, ulice Vídeňská"
1,Costa Express - Mělník,50.34802581272294,14.492174487249713,"ČS Shell - Mělník, ulice Mladoboleslavská"
2,Costa Express - Praha 6,50.093505335502975,14.331827076074205,"ČS Shell - Praha 6, ulice Evropská"
3,Costa Express - Rokycany,49.7538998,13.5894548,"ČS Shell - Rokycany, dálnice D5 (směr Praha)"
4,Costa Express - Tábor,49.41056965127617,14.683676139550812,"ČS Shell - Tábor, ulice Chýnovská"


In [18]:
def df_to_json(df):
    express_json = df.to_json(orient='records')
    temp_po = json.loads(express_json, encoding='utf-8')
    temp_json = json.dumps(temp_po, ensure_ascii = False)
    return temp_json

In [20]:
b = df_to_json(expressDf)

In [21]:
def print_json(json):
    return print(json)

In [22]:
print_json(b)

[{"title": "Costa Express - Praha 4", "lat": "50.034589163874585", "lng": "14.455249451931763", "address": "ČS Shell - Praha 4, ulice Vídeňská"}, {"title": "Costa Express - Mělník", "lat": "50.34802581272294", "lng": "14.492174487249713", "address": "ČS Shell - Mělník, ulice Mladoboleslavská"}, {"title": "Costa Express - Praha 6", "lat": "50.093505335502975", "lng": "14.331827076074205", "address": "ČS Shell - Praha 6, ulice Evropská"}, {"title": "Costa Express - Rokycany", "lat": "49.7538998", "lng": "13.5894548", "address": "ČS Shell - Rokycany, dálnice D5 (směr Praha)"}, {"title": "Costa Express - Tábor", "lat": "49.41056965127617", "lng": "14.683676139550812", "address": "ČS Shell - Tábor, ulice Chýnovská"}, {"title": "Costa Express - Velké Meziříčí", "lat": "49.35369844640703", "lng": "16.033142081103506", "address": "ČS Shell - Velké Meziříčí, dálnice D1 (směr Brno – Praha)"}, {"title": "Costa Express - Praha", "lat": "50.1089157", "lng": "14.6410117", "address": "ČS Shell - Dáln